In [1]:
import random, os, tqdm, time, json
import numpy as np
import pandas as pd
from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt

import sys
sys.path.append("../../../../")

random.seed(618)
np.random.seed(907)

new_base_path = os.path.join(
    "/Users/minkexiu/Downloads/",
    "/".join(
        os.getcwd().split("/")[-1*(len(sys.path[-1].split("/")) - 1):]
    ),
)
print("storage dir:", new_base_path)
print("code dir:", os.getcwd())

## 创建文件夹。
if not os.path.exists(new_base_path):
    os.makedirs(
        new_base_path
    )
if not os.path.exists(os.path.join(new_base_path, "preprocessedData")):
    os.makedirs(
        os.path.join(new_base_path, "preprocessedData")
    )
if not os.path.exists(os.path.join(new_base_path, "originalData")):
    os.makedirs(
        os.path.join(new_base_path, "originalData")
    )
if not os.path.exists(os.path.join(new_base_path, "trained_models")):
    os.makedirs(
        os.path.join(new_base_path, "trained_models")
    )

def create_originalData_path(filename_or_path):
    return os.path.join(new_base_path, "originalData", filename_or_path)
def create_preprocessedData_path(filename_or_path):
    return os.path.join(new_base_path, "preprocessedData", filename_or_path)
def create_trained_models_path(filename_or_path):
    return os.path.join(new_base_path, "trained_models", filename_or_path)

def millisec2datetime(timestamp):
    time_local = time.localtime(timestamp/1000)
    return time.strftime("%Y-%m-%d %H:%M:%S", time_local)
    
def run_finish():
    # 假设你的字体文件是 'myfont.ttf' 并且位于当前目录下  
    font = FontProperties(fname="/Users/minkexiu/Documents/GitHub/ML_Tryout/SimHei.ttf", size=24)  
    # 创建一个空白的图形  
    fig, ax = plt.subplots()  
    ax.imshow(
        plt.imread("/Users/minkexiu/Downloads/wallhaven-dgxpyg.jpg")
    )
    # 在图形中添加文字  
    ax.text(
        ax.get_xlim()[1] * 0.5, 
        ax.get_ylim()[0] * 0.5, 
        f"程序于这个点跑完：\n{millisec2datetime(time.time()*1000)}", fontproperties=font, ha="center", va="center", color="red"
    )  
    # 设置图形的布局  
    # ax.set_xlim(0, 1)  
    # ax.set_ylim(0, 1)  
    ax.set_xticks([])  
    ax.set_yticks([])  
    ax.patch.set_color("blue")
    # 显示图形  
    plt.show()
        
tqdm.tqdm.pandas() ## 引入这个，就可以在apply的时候用progress_apply了。

import IPython
def kill_current_kernel():
    '''杀死当前的kernel释放内存空间。'''
    IPython.Application.instance().kernel.do_shutdown(True) 
    
def simply_show_data(df1):
    print(df1.shape)
    display(df1.head())
    
def wait_flag(saved_flag_path, time_interval_sec=10):
    print("waiting for", saved_flag_path)
    time_count = 0
    while True:
        if os.path.exists(saved_flag_path):
            break
        time.sleep(time_interval_sec)
        time_count+=time_interval_sec
        print(time_count, end=" ")
    print("finish!!")

storage dir: /Users/minkexiu/Downloads/GitHub/ML_runCodeFromBook/大规模语言模型：从理论到实践/RLHF_PPO
code dir: /Users/minkexiu/Documents/GitHub/ML_runCodeFromBook/大规模语言模型：从理论到实践/RLHF_PPO


# (之前的草稿)

In [2]:
## 加载acm的tokenizer，
## 加载一下数据试试看。

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(create_trained_models_path("Qwen1.5-0.5B-Chat"))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
tokenizer

Qwen2TokenizerFast(name_or_path='/Users/minkexiu/Downloads/GitHub/ML_runCodeFromBook/大规模语言模型：从理论到实践/RLHF_PPO/trained_models/Qwen1.5-0.5B-Chat', vocab_size=151643, model_max_length=32768, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [8]:
## 加载一下数据：
with open(
    "/Users/minkexiu/Documents/GitHub/ML_runCodeFromBook/大规模语言模型：从理论到实践/RLHF_PPO/data/train_data.json", 
    'r', 
    encoding="utf-8"
) as f:
    data = json.load(f)

In [9]:
data
## 数据大概长这样。有一个query，有一个system_content。

[{'query': '饭店服务员的态度太差，使用委婉积极的态度投诉', 'system_content': '你是一个有文化的文明人'},
 {'query': '领导故意刁难你，你想骂他娘的，使用文明语言骂他娘的', 'system_content': '你是一个有文化的文明人'}]

# 尝试一下呗。

In [4]:
import torch

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [8]:
from peft import LoraConfig, get_peft_model, PeftModel

# 初始化一个演员·评论员模型

## 演员·评论员

In [18]:
## 尝试初始化一下原始的文本生成模型.
## 这个模型是一个生成模型哦。
model = AutoModelForCausalLM.from_pretrained(
    create_trained_models_path("Qwen1.5-0.5B-Chat")
).to("cpu").eval()

In [24]:
tokenizer = AutoTokenizer.from_pretrained(create_trained_models_path("Qwen1.5-0.5B-Chat"))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [76]:
## 准备数据吧。
messages = [
    {"role": "system", "content": "你是一个有文化的文明人"},
    {"role": "user", "content": "饭店服务员的态度太差，使用委婉积极的态度投诉"},
]

In [29]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

In [30]:
text

'<|im_start|>system\n你是一个有文化的文明人<|im_end|>\n<|im_start|>user\n饭店服务员的态度太差，使用委婉积极的态度投诉<|im_end|>\n<|im_start|>assistant\n'

In [36]:
tokenizer?

Signature:     
tokenizer(
    text: Union[str, List[str], List[List[str]]] = None,
    text_pair: Union[str, List[str], List[List[str]], NoneType] = None,
    text_target: Union[str, List[str], List[List[str]]] = None,
    text_pair_target: Union[str, List[str], List[List[str]], NoneType] = None,
    add_special_tokens: bool = True,
    padding: Union[bool, str, transformers.utils.generic.PaddingStrategy] = False,
    truncation: Union[bool, str, transformers.tokenization_utils_base.TruncationStrategy] = None,
    max_length: Optional[int] = None,
    stride: int = 0,
    is_split_into_words: bool = False,
    pad_to_multiple_of: Optional[int] = None,
    return_tensors: Union[str, transformers.utils.generic.TensorType, NoneType] = None,
    return_token_type_ids: Optional[bool] = None,
    return_attention_mask: Optional[bool] = None,
    return_overflowing_tokens: bool = False,
    return_special_tokens_mask: bool = False,
    return_offsets_mapping: bool = False,
    return_length:

In [31]:
model_inputs = tokenizer([text], return_tensors="pt")

In [32]:
model_inputs

{'input_ids': tensor([[151644,   8948,    198,  56568, 101909,  18830, 107705, 100704,  17340,
         151645,    198, 151644,    872,    198, 107514, 112822, 105421,  99222,
          99572,   3837,  37029,  99199, 106783,  99666, 105421, 104943, 151645,
            198, 151644,  77091,    198]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])}

In [37]:
[
    model_inputs.input_ids.tolist()[0], 
    model_inputs.attention_mask.tolist()[0]
]

[[151644,
  8948,
  198,
  56568,
  101909,
  18830,
  107705,
  100704,
  17340,
  151645,
  198,
  151644,
  872,
  198,
  107514,
  112822,
  105421,
  99222,
  99572,
  3837,
  37029,
  99199,
  106783,
  99666,
  105421,
  104943,
  151645,
  198,
  151644,
  77091,
  198],
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]]

## Lora

In [11]:
from dataclasses import dataclass, field

In [16]:
lora_config = LoraConfig(
    r=2, ## 把秩降到这个数。
    lora_alpha=8, ## 这个是一个扩张系数。
    target_modules=['k_proj',  'v_proj'],
    lora_dropout=0,
    task_type="CAUSAL_LM",
)

In [22]:
lora_model = PeftModel(model, lora_config)
v_head = torch.nn.Linear(1024, 1, bias=False).to("cpu")
for name, module in lora_model.named_modules():
    if 'lora_' in name:
        for param in module.parameters():
            param.requires_grad = True

In [40]:
# def lora_forward(input_ids, attention_mask, tools):
#     res = lora_model.forward(input_ids, attention_mask, output_hidden_states=True)
#     values = v_head(res.hidden_states[0]).squeeze(-1)[:, :-1]

In [55]:
tokenizer.batch_decode(model_inputs.input_ids)

['<|im_start|>system\n你是一个有文化的文明人<|im_end|>\n<|im_start|>user\n饭店服务员的态度太差，使用委婉积极的态度投诉<|im_end|>\n<|im_start|>assistant\n']

In [41]:
generated_ids = lora_model.generate(model_inputs.input_ids, max_new_tokens=512, top_p=1.0,
                                            num_beams=1,
                                            do_sample=False)

In [44]:
generated_ids.shape

torch.Size([1, 224])

In [43]:
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True) ## 这个就是把id解码成自然语言字符。
response

['system\n你是一个有文化的文明人\nuser\n饭店服务员的态度太差，使用委婉积极的态度投诉\nassistant\n尊敬的经理：\n\n您好！我最近在您的饭店用餐时遇到了一些问题。我想通过这封信向您反映一下。\n\n首先，我对餐厅的服务态度感到非常不满。当我点菜时，服务员的态度并不友好，总是显得有些冷漠和不耐烦。他们似乎对我的需求没有足够的关注，甚至有时会对我提出一些不合理的建议或要求。\n\n其次，我在用餐过程中也遇到了一些困扰。我发现有些菜品的味道并不符合我的口味，而且有些服务人员的服务态度也不够热情。这些都让我感到很失望。\n\n最后，我还注意到有一些环境问题。例如，餐厅的卫生状况不佳，餐具和杯子经常被污染，这让我感到非常不舒服。\n\n我希望您能理解并采取措施来改善我们的用餐体验。我相信，只要我们共同努力，我们的服务质量将会得到显著提高。\n\n再次感谢您抽出宝贵的时间阅读这封信，并期待您的回复。\n\n顺祝商祺，\n[你的名字]']

In [58]:
response_id = generated_ids[:, model_inputs.input_ids.shape[1]:]

In [60]:
generated_ids.shape

torch.Size([1, 224])

In [62]:
response

['system\n你是一个有文化的文明人\nuser\n饭店服务员的态度太差，使用委婉积极的态度投诉\nassistant\n尊敬的经理：\n\n您好！我最近在您的饭店用餐时遇到了一些问题。我想通过这封信向您反映一下。\n\n首先，我对餐厅的服务态度感到非常不满。当我点菜时，服务员的态度并不友好，总是显得有些冷漠和不耐烦。他们似乎对我的需求没有足够的关注，甚至有时会对我提出一些不合理的建议或要求。\n\n其次，我在用餐过程中也遇到了一些困扰。我发现有些菜品的味道并不符合我的口味，而且有些服务人员的服务态度也不够热情。这些都让我感到很失望。\n\n最后，我还注意到有一些环境问题。例如，餐厅的卫生状况不佳，餐具和杯子经常被污染，这让我感到非常不舒服。\n\n我希望您能理解并采取措施来改善我们的用餐体验。我相信，只要我们共同努力，我们的服务质量将会得到显著提高。\n\n再次感谢您抽出宝贵的时间阅读这封信，并期待您的回复。\n\n顺祝商祺，\n[你的名字]']

In [63]:
response_id.shape

torch.Size([1, 193])

In [64]:
prompt_generate = response
prompt_generate

['system\n你是一个有文化的文明人\nuser\n饭店服务员的态度太差，使用委婉积极的态度投诉\nassistant\n尊敬的经理：\n\n您好！我最近在您的饭店用餐时遇到了一些问题。我想通过这封信向您反映一下。\n\n首先，我对餐厅的服务态度感到非常不满。当我点菜时，服务员的态度并不友好，总是显得有些冷漠和不耐烦。他们似乎对我的需求没有足够的关注，甚至有时会对我提出一些不合理的建议或要求。\n\n其次，我在用餐过程中也遇到了一些困扰。我发现有些菜品的味道并不符合我的口味，而且有些服务人员的服务态度也不够热情。这些都让我感到很失望。\n\n最后，我还注意到有一些环境问题。例如，餐厅的卫生状况不佳，餐具和杯子经常被污染，这让我感到非常不舒服。\n\n我希望您能理解并采取措施来改善我们的用餐体验。我相信，只要我们共同努力，我们的服务质量将会得到显著提高。\n\n再次感谢您抽出宝贵的时间阅读这封信，并期待您的回复。\n\n顺祝商祺，\n[你的名字]']

In [65]:
prompt_generate_ids = generated_ids
prompt_generate_ids

tensor([[151644,   8948,    198,  56568, 101909,  18830, 107705, 100704,  17340,
         151645,    198, 151644,    872,    198, 107514, 112822, 105421,  99222,
          99572,   3837,  37029,  99199, 106783,  99666, 105421, 104943, 151645,
            198, 151644,  77091,    198, 109723,   9370, 100249,  48443, 111308,
           6313,  35946, 104044,  18493, 101214, 107514, 111554,  13343, 109075,
         101883,  86119,   1773, 104100,  67338,  43288,  99690,  21317,  69041,
          87026, 101279, 100158,   3407, 101140,   3837, 108531, 104835, 105646,
         102316, 104048,  99491, 106974,   1773, 108089,  27442,  99800,  13343,
           3837, 112822, 105421, 100684, 106098,   3837, 104014, 104392, 101895,
         113808,  33108,  16530, 100796, 100886,   1773,  99650, 101994,  32664,
          97611, 100354,  80443, 103170, 100020,   3837, 100636, 104685,  36993,
         102788, 101080, 101883,  16530, 105630, 101898,  57191, 101882,   3407,
         102460,   3837, 104

In [66]:
generate_ids = response_id
generate_ids.shape

torch.Size([1, 193])

In [67]:
attention_mask = (prompt_generate_ids != tokenizer.pad_token_id)

In [68]:
attention_mask

tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, T

In [70]:
generate_ids.shape

torch.Size([1, 193])

In [72]:
tokenizer.batch_decode(generate_ids[:, -1:])

['<|im_end|>']

In [73]:
generate_ids_mask = (generate_ids[
                     :, 
                     :-1 ## 这里省掉了最后的结尾符。
                     ] != tokenizer.pad_token_id)
generate_ids_mask

tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, T

In [74]:
response_shape = generate_ids.shape[1] - 1

In [77]:
tools = Tools(response_shape, generate_ids_mask)
【ckpt】main.py 代码的44行。莫忘莫忘。

SyntaxError: invalid character '【' (U+3010) (111120032.py, line 2)